In [2]:
import numpy as np

In [24]:
def svd(A):
    eigvalsv, V = np.linalg.eigh(A.T @ A)
    idx = np.argsort(eigvalsv)[::-1]
    eigvalsv = eigvalsv[idx]
    V = V[:, idx]
    sigma = np.sqrt(eigvalsv)
    Sigma = np.diag(sigma)
    eigvalsu, U = np.linalg.eigh(A @ A.T)
    eigvalsu = eigvalsu[idx]
    U = U[:, idx]
    return U @ Sigma @ V.T

A=np.array([[1,2,0],[2,4,5],[3,1,7]])
np.round(svd(A))

array([[-1., -2., -0.],
       [-2., -4., -5.],
       [-3., -1., -7.]])